# <center> Run custom Spark on Palmetto instead of Cypress </center>

# Required Modules 
* hdp/0.1
* anaconda3/5.1.0

In [1]:
!module list 

Currently Loaded Modulefiles:
  1) gcc/7.1.0         3) use.own           5) hdp/0.1
  2) vim/8.0.1763      4) anaconda3/5.1.0


In [2]:
import sys
print(sys.version_info)
assert(sys.version.split('|')[0].replace(' ','') == '3.6.4'), "Wrong python version for pyspark"

sys.version_info(major=3, minor=6, micro=4, releaselevel='final', serial=0)


In [3]:
!cypress-kinit
!klist

Ticket cache: FILE:/home/esadrfa/.krb5cc
Default principal: esadrfa@PALMETTO.CLEMSON.EDU

Valid starting       Expires              Service principal
07/30/2018 20:57:41  08/06/2018 20:57:41  krbtgt/PALMETTO.CLEMSON.EDU@PALMETTO.CLEMSON.EDU


## 1. Getting Started

Spark stores data in memory. This memory space is represented by variable **sc** (SparkContext). 

In [4]:
# import sys
# import os

# sys.path.insert(0, '/usr/hdp/current/spark2-client/python')
# sys.path.insert(0, '/usr/hdp/current/spark2-client/python/lib/py4j-0.10.6-src.zip')

# os.environ['SPARK_HOME'] = '/usr/hdp/current/spark2-client/'
# os.environ['SPARK_CONF_DIR'] = '/etc/hadoop/synced_conf/spark2/'
# os.environ['PYSPARK_PYTHON'] = '/software/anaconda3/5.1.0/bin/python'

# import pyspark
# conf = pyspark.SparkConf()
# conf.setMaster("yarn")
# conf.set("spark.driver.memory","1200g")  #request a little less than what we asked from palmetto (6g)
# conf.set("spark.driver.maxResultSize", "120g")
# conf.set("spark.executor.memory","160g")   # 60g per instance of executer
# conf.set("spark.executor.instances", "40")
# conf.set("spark.executor.cores","10")       # more than 5 bug down the HDFS
# conf.set("spark.rpc.message.maxSize", "1024")
# conf.set("spark.driver.maxResultSize", "600g")

# sc = pyspark.SparkContext(conf=conf)

In [5]:
import sys
import os

sys.path.insert(0, '/usr/hdp/current/spark2-client/python')
sys.path.insert(0, '/usr/hdp/current/spark2-client/python/lib/py4j-0.10.6-src.zip')

os.environ['SPARK_HOME'] = '/usr/hdp/current/spark2-client/'
os.environ['SPARK_CONF_DIR'] = '/etc/hadoop/synced_conf/spark2/'
os.environ['PYSPARK_PYTHON'] = '/software/anaconda3/5.1.0/bin/python'

# https://spoddutur.github.io/spark-notes/distribution_of_executors_cores_and_memory_for_spark_application.html
    
import pyspark
conf = pyspark.SparkConf()
conf.setMaster("yarn")
conf.set("spark.driver.memory","1200g")  #request a little less than what we asked from palmetto (6g)
conf.set("spark.driver.maxResultSize", "120g")
conf.set("spark.executor.memory","80g")   # 60g per instance of executer
conf.set("spark.executor.instances", "480")
conf.set("spark.executor.cores","5")       # more than 5 bug down the HDFS
conf.set("spark.rpc.message.maxSize", "1024")
conf.set("spark.driver.maxResultSize", "300g")

sc = pyspark.SparkContext(conf=conf)

In [ ]:
sc

In [31]:
sc.stop()

In [6]:
print(sc.version)
sqlContext = pyspark.SQLContext(sc)

2.3.0.2.6.5.0-292


In [7]:
from multiprocessing.dummy import Pool as ThreadPool 
import pickle
import numpy as np
from pyspark.ml.linalg import SparseVector

# HDFS Data

In [7]:
data_path = 'HypoClas/'
scaled_df_folder = '/SO_CS_EU_s0_e99999_scaled/'
test_folder = '/SO_s0_e9999_scaled/'

In [7]:
# !hdfs dfs -ls HypoClas/

# Python scipy matrix

In [6]:
import pickle
# featurePath = '/scratch3/esadrfa/web_scratch/all/trained_files_2017/'
featurePath = '/scratch2/esadrfa/perceptron/data/'
mat_filename = 'SO_CS_EU_scaled_s0_e99999_csr_part'


In [7]:
!ls -lh {featurePath}

total 188G
-rw-rw-r-- 1 esadrfa cuuser  21G Jul  1 16:59 SO_CS_EU_scaled_s0_e99999_csr_part0.p
-rw-rw-r-- 1 esadrfa cuuser  18G Jul  1 17:22 SO_CS_EU_scaled_s0_e99999_csr_part1.p
-rw-rw-r-- 1 esadrfa cuuser  16G Jul  1 17:38 SO_CS_EU_scaled_s0_e99999_csr_part2.p
-rw-rw-r-- 1 esadrfa cuuser  15G Jul  1 17:49 SO_CS_EU_scaled_s0_e99999_csr_part3.p
-rw-rw-r-- 1 esadrfa cuuser  13G Jul  1 18:00 SO_CS_EU_scaled_s0_e99999_csr_part4.p
-rw-r--r-- 1 esadrfa cuuser 108G Jul 26 20:26 SO_CS_EU_Transposed_scaled_s0_e99999_csr.p


In [12]:
num_range=5

In [13]:
lst_mat_parts = [None] * num_range
selected_indices = [i for i in range(num_range)]

In [16]:
def loadPart(idx):
    lst_mat_parts[idx] = pickle.load(open(featurePath + mat_filename + str(idx) +'.p' , 'rb'))

In [17]:
%%time

pool = ThreadPool() 
pool.map(loadPart, selected_indices)

pool.close()
pool.join()

CPU times: user 55.2 s, sys: 2min 5s, total: 3min
Wall time: 1min 49s


## merge parts and transpose them

In [18]:
len(lst_mat_parts)

5

#### merge matrix parts to one matrix 

In [19]:
%%time
from scipy.sparse import vstack

fullMatrix = vstack(lst_mat_parts)

CPU times: user 5min 19s, sys: 4min 31s, total: 9min 50s
Wall time: 9min 43s


In [20]:
fullMatrix.shape

(100000, 23058100)

In [21]:
%%time
fullMatrix = fullMatrix.transpose()

CPU times: user 52.1 s, sys: 8.14 s, total: 1min
Wall time: 59.3 s


In [23]:
%%time
fullMatrix = fullMatrix.tocsr()

CPU times: user 3min 34s, sys: 2min 7s, total: 5min 41s
Wall time: 5min 39s


In [ ]:
# 072618-1925 I run the above cells, but it is not clear how to convert the data to RowMatrix.
# I continue with Custom_spark_SVD_Cypress notebook

In [24]:
type(fullMatrix)

scipy.sparse.csr.csr_matrix

In [25]:
fullMatrix

(23058100, 100000)

In [26]:
%%time
import pickle
pickle.dump(fullMatrix\
            , open(featurePath + 'SO_CS_EU_Transposed_scaled_s0_e99999_csr.p', 'wb')\
            , protocol=4)

CPU times: user 2min 23s, sys: 1h 33min 30s, total: 1h 35min 54s
Wall time: 48min 56s


## Load after crashed Kernel 072718-1645

In [8]:
%%time
import pickle
fullMatrix = pickle.load(open(featurePath + 'SO_CS_EU_Transposed_scaled_s0_e99999_csr.p', 'rb'))

CPU times: user 1min 17s, sys: 2min 35s, total: 3min 52s
Wall time: 3min 52s


## Convert the transposed csr_matrix to RowMatrix

### Create a list of sparseVectors which
* first used to make dataframe 
* second the dataframe is converted to RowMatrix

In [11]:
num_row, num_col = fullMatrix.shape
print(num_row, num_col)

lst_total = [None] * num_row
selected_indices = [i for i in range(num_row)]

def addMllibSparseVector(idx):
    curr = fullMatrix.getrow(idx)
    arr_ind = np.argsort(curr.indices)
    lst_total[idx] = (idx, SparseVector(num_col\
                 , curr.indices[arr_ind], curr.data[arr_ind]),)

23058100 100000


In [15]:
%%time
from multiprocessing.dummy import Pool as ThreadPool 
pool = ThreadPool() 
pool.map(addMllibSparseVector, selected_indices)

pool.close()
pool.join()

CPU times: user 5h 5min 53s, sys: 2h 59min 49s, total: 8h 5min 43s
Wall time: 4h 16min 14s


In [16]:
import math
batch_size = 5000
num_range = math.ceil(num_row / batch_size)
print(num_range)

4612


In [21]:
report_interval = int(num_range / 1000)
report_interval

4

In [18]:
data_path = 'HypoClas/'
scaled_transposed_df_parts_folder = '/SO_CS_EU_s0_e99999_scaled_transposed_part'

In [19]:
num_row

23058100

In [ ]:
lst_dfs = [None] * num_range
selected_dataframes = [i for i in range(num_range)]

def makeDataframes(idx):
    start = idx * batch_size
    end = min(start + batch_size, num_row)
    lst_dfs[idx] = sqlContext.createDataFrame(\
                      lst_total[start:end], ["id", "features"])

In [ ]:
%%time
pool = ThreadPool() 
pool.map(makeDataframes, selected_dataframes)
pool.close()
pool.join()

In [22]:
%%time
for idx in range(num_range):
    if not idx % report_interval:
        print(idx, end=',')
    start = idx * batch_size
    end = min(start + batch_size, num_row)
    df = sqlContext.createDataFrame(lst_total[start:end], ["id", "features"])
    df.write.save(data_path + scaled_transposed_df_parts_folder + str(idx), format='parquet', mode='overwrite')

0,4,8,12,16,20,24,28,32,36,40,44,48,52,56,60,64,68,72,76,80,84,88,92,96,100,104,108,112,116,120,124,128,132,136,140,144,148,152,156,160,164,168,172,176,180,184,188,192,196,200,204,208,212,216,220,224,228,232,236,240,244,248,252,256,260,264,268,272,276,280,284,288,292,296,300,304,308,312,316,320,324,328,332,336,340,344,348,352,356,360,364,368,372,376,380,384,388,392,396,400,404,408,412,416,420,424,428,432,436,440,444,448,452,456,460,464,468,472,476,480,484,488,492,496,500,504,508,512,516,520,524,528,532,536,540,544,548,552,556,560,564,568,572,576,580,584,588,592,596,600,604,608,612,616,620,624,628,632,636,640,644,648,652,656,660,664,668,672,676,680,684,688,692,696,700,704,708,712,716,720,724,728,732,736,740,744,748,752,756,760,764,768,772,776,780,784,788,792,796,800,804,808,812,816,820,824,828,832,836,840,844,848,852,856,860,864,868,872,876,880,884,888,892,896,900,904,908,912,916,920,924,928,932,936,940,944,948,952,956,960,964,968,972,976,980,984,988,992,996,1000,1004,1008,1012,1016,102

## the connection to HPC failed over the weekend, I restart the loading be initializing the required variables

In [8]:
num_range = 4612
data_path = 'HypoClas/'
scaled_transposed_df_parts_folder = '/SO_CS_EU_s0_e99999_scaled_transposed_part'
report_interval = int(num_range / 100)
report_interval

46

In [9]:
lst_dfs = [None] * num_range
selected_indices_dfs = [i for i in range(num_range)]

def loadDataFrame(idx):
    lst_dfs[idx] = sqlContext.read.format('parquet')\
               .load(data_path + scaled_transposed_df_parts_folder + str(idx))

In [10]:
%%time
pool = ThreadPool() 
pool.map(loadDataFrame, selected_indices_dfs)

pool.close()
pool.join()

CPU times: user 2.7 s, sys: 569 ms, total: 3.27 s
Wall time: 1min 58s


In [11]:
from functools import reduce  # For Python 3.x
from pyspark.sql import DataFrame

# https://datascience.stackexchange.com/a/11361/8723
def unionAll(*dfs):
    return reduce(DataFrame.unionAll, dfs)

In [12]:
%%time
raw_df = unionAll(*lst_dfs)

CPU times: user 1.63 s, sys: 312 ms, total: 1.95 s
Wall time: 5min 54s


In [13]:
print(raw_df.columns)
print(type(raw_df))

['id', 'features']
<class 'pyspark.sql.dataframe.DataFrame'>


In [14]:
%%time
# https://stackoverflow.com/a/37286339/2674061
raw_rdd = raw_df.select('features').rdd.map(list)

CPU times: user 25.3 ms, sys: 8.37 ms, total: 33.7 ms
Wall time: 2min 9s


In [15]:
%%time
raw_rdd.cache()

CPU times: user 5.84 ms, sys: 1.39 ms, total: 7.23 ms
Wall time: 31.1 ms


PythonRDD[18451] at RDD at PythonRDD.scala:48

In [16]:
%%time
from pyspark.mllib.linalg.distributed import IndexedRow, IndexedRowMatrix, RowMatrix
from pyspark.ml.linalg import SparseVector

mat = RowMatrix(raw_rdd)

CPU times: user 32.9 ms, sys: 6.56 ms, total: 39.5 ms
Wall time: 136 ms


In [17]:
%%time
# Compute the top k singular values and corresponding singular vectors.
svd = mat.computeSVD(1000, computeU=True)

Py4JJavaError: An error occurred while calling o18519.computeSVD.
: org.apache.spark.SparkException: Job aborted due to stage failure: Total size of serialized results of 457487 tasks (300.0 GB) is bigger than spark.driver.maxResultSize (300.0 GB)
	at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:1599)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1587)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1586)
	at scala.collection.mutable.ResizableArray$class.foreach(ResizableArray.scala:59)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:48)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:1586)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$handleTaskSetFailed$1.apply(DAGScheduler.scala:831)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$handleTaskSetFailed$1.apply(

In [17]:
mat